1 THUCNews数据集与预处理

    1.1 数据集下载
    1.2 数据量
    1.3 数据预处理
        1.3.1 导入数据
        1.3.2 将文字型的label 转为数字label
        
2 特征工程

    2.1 Jieba分词
    2.2 TF-IDF并将文件保存至本地
3 训练模型

    3.1 lightgbm模型
    
4 K 折交叉验证


### IMDB数据集下载和探索

In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

SystemError: <class '_frozen_importlib._ModuleLockManager'> returned a result with an error set

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

In [ ]:
imdb = keras.datasets.imdb   #下载 IMDB 数据集

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

### 探索数据

In [ ]:
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))

In [ ]:
len(train_data[0]), len(train_data[1])

#### 创建一个辅助函数来查询包含整数到字符串映射的字典对象：

In [ ]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

#### 使用 decode_review 函数显示第一条影评的文本：

In [ ]:
decode_review(train_data[0])

#### 准备数据

#### 影评（整数数组）必须转换为张量，然后才能馈送到神经网络中。我们可以通过以下两种方法实现这种转换：
1.对数组进行独热编码，将它们转换为由 0 和 1 构成的向量。例如，序列 [3, 5] 将变成一个 10000 维的向量，除索引 3 和 5 转换为 1 之外，其余全转换为 0。然后，将它作为网络的第一层，一个可以处理浮点向量数据的密集层。不过，这种方法会占用大量内存，需要一个大小为 num_words * num_reviews 的矩阵。

2.或者，我们可以填充数组，使它们都具有相同的长度，然后创建一个形状为 max_length * num_reviews 的整数张量。我们可以使用一个能够处理这种形状的嵌入层作为网络中的第一层。


#### 使用第二种方法

#### 由于影评的长度必须相同，我们将使用 pad_sequences 函数将长度标准化：

In [ ]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

#### 看看样本的长度：

In [ ]:
len(train_data[0]), len(train_data[1])

In [ ]:
#并检查（现已填充的）第一条影评：

print(train_data[0])

#### 构建模型

In [ ]:
# input shape is the vocabulary count used for the movie reviews (10,000 words)
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

#### 按顺序堆叠各个层以构建分类器：

第一层是 Embedding 层。该层会在整数编码的词汇表中查找每个字词-索引的嵌入向量。模型在接受训练时会学习这些向量。这些向量会向输出数组添加一个维度。生成的维度为：(batch, sequence, embedding)。
接下来，一个 GlobalAveragePooling1D 层通过对序列维度求平均值，针对每个样本返回一个长度固定的输出向量。这样，模型便能够以尽可能简单的方式处理各种长度的输入。
该长度固定的输出向量会传入一个全连接 (Dense) 层（包含 16 个隐藏单元）。
最后一层与单个输出节点密集连接。应用 sigmoid 激活函数后，结果是介于 0 到 1 之间的浮点值，表示概率或置信水平。

#### 隐藏单元
上述模型在输入和输出之间有两个中间层（也称为“隐藏”层）。输出（单元、节点或神经元）的数量是相应层的表示法空间的维度。换句话说，该数值表示学习内部表示法时网络所允许的自由度。

如果模型具有更多隐藏单元（更高维度的表示空间）和/或更多层，则说明网络可以学习更复杂的表示法。不过，这会使网络耗费更多计算资源，并且可能导致学习不必要的模式（可以优化在训练数据上的表现，但不会优化在测试数据上的表现）。这称为过拟合，我们稍后会加以探讨。

#### 损失函数和优化器
模型在训练时需要一个损失函数和一个优化器。由于这是一个二元分类问题且模型会输出一个概率（应用 S 型激活函数的单个单元层），因此我们将使用 binary_crossentropy 损失函数。

该函数并不是唯一的损失函数，例如，您可以选择 mean_squared_error。但一般来说，binary_crossentropy 更适合处理概率问题，它可测量概率分布之间的“差距”，在本例中则为实际分布和预测之间的“差距”。

稍后，在探索回归问题（比如预测房价）时，我们将了解如何使用另一个称为均方误差的损失函数。

现在，配置模型以使用优化器和损失函数：

In [ ]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

#### 创建验证集
在训练时，我们需要检查模型处理从未见过的数据的准确率。我们从原始训练数据中分离出 10000 个样本，创建一个验证集。（为什么现在不使用测试集？我们的目标是仅使用训练数据开发和调整模型，然后仅使用一次测试数据评估准确率。）

In [ ]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

#### 训练模型
用有 512 个样本的小批次训练模型 40 个周期。这将对 x_train 和 y_train 张量中的所有样本进行 40 次迭代。在训练期间，监控模型在验证集的 10000 个样本上的损失和准确率：

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

#### 评估模型

In [ ]:
results = model.evaluate(test_data, test_labels)

print(results)

#### 创建准确率和损失随时间变化的图
model.fit() 返回一个 History 对象，该对象包含一个字典，其中包括训练期间发生的所有情况：

In [ ]:
history_dict = history.history
history_dict.keys()

#### 一共有 4 个条目：每个条目对应训练和验证期间的一个受监控指标。我们可以使用这些指标绘制训练损失与验证损失图表以进行对比，并绘制训练准确率与验证准确率图表：

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # clear figure
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

### THUCNews数据集下载和探索

In [ ]:
import pandas as pd
train_data=pd.read_csv('cnews_train.txt',sep='\t',names=['label','content'])
test_data=pd.read_csv('cnews.test.txt',sep='\t',names=['content'])
train_data.info()


### 预处理

### 将文字型的label 转为数字label

In [ ]:
def read_category(y_train):
    """读取分类目录，固定"""
    categories = ['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐']
    categories = [x for x in categories]
    cat_to_id = dict(zip(categories, range(len(categories))))
    label_id = []
    for i in range(len(y_train)):
        label_id.append(cat_to_id[y_train[i]])
    return label_id
    
train_target=train_data['label']  
y_label=read_category(train_target)


In [ ]:
特征工程

In [ ]:
def chinese_word_cut(mytext):  
    return " ".join(jieba.cut(mytext))
# 不添加分词
#train_content = train_data['content']
#test_content = test_data['content']

# 添加分词
train_content =train_data['content'].apply(chinese_word_cut)
test_content = test_data['content'].apply(chinese_word_cut)


#### TF-IDF并将文件保存至本地

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
f_all = pd.concat(objs=[train_data['content'], test_data['content']], axis=0)
tfidf_vect = TfidfVectorizer(max_df = 0.9,min_df = 3,token_pattern=r"(?u)\b\w+\b")
tfidf_vect.fit(f_all)
X_train=tfidf_vect.fit_transform(train_data['content'])
X_test=tfidf_vect.fit_transform(test_data['content'])


In [ ]:
import pickle
data = (X_train, y_label, X_test)
fp = open('data_tfidf.pkl', 'wb')
pickle.dump(data, fp)
fp.close()

#### 训练模型

#### lightgbm模型

In [ ]:
import lightgbm as lgb
X_trn, X_val, y_trn, y_val= train_test_split(X_train,y_label,test_size=0.2, random_state=2019)
train_data=lgb.Dataset(X_trn,label=y_trn)
validation_data=lgb.Dataset(X_val,label=y_val)

params= {
    'objective':'multiclass',
    'num_class':10, 
    'boosting': 'gbdt',
    'num_threads': 1,
    'learning_rate': 0.3,
    'num_leaves': 31, 
    'max_depth': 8,
  
    'max_bin':200,
    'lambda_l1': 0,
    'lambda_l2': 0,
    }

clf=lgb.train(params,train_data,valid_sets=[validation_data])


### 召回率、准确率、ROC曲线、AUC、PR曲线基本概念

In [ ]:
精确率和召回率
精确率，又称查准率(Precision)：正样本的预测数/被预测为正样本的数量（注意：精确率和准确率不同），即：
P=TP/(TP+FP) P=TP/(TP+FP)
P=TP/(TP+FP)

召回率(Recall)又称查全率：分类正确的正样本个数占正样本个数的比例，即：
R=TPTP+FN R=TPTP+FN
R=TPTP+FN


In [ ]:
ROC曲线常用于二分类问题中的模型比较，主要表现为一种真正例率 (TPR) 和假正例率 (FPR) 的权衡。

具体方法是在不同的分类阈值 (threshold) 设定下分别以TPR和FPR为纵、横轴作图。由ROC曲线的两个指标，
TPR=TP/P=TP/(TP+FN) TPR=TP/P=TP/(TP+FN)
TPR=TP/P=TP/(TP+FN)
，
FPR=FP/N=FP/(FP+TN) FPR=FP/N=FP/(FP+TN)
FPR=FP/N=FP/(FP+TN)
可以看出：

当一个样本被分类器判为正例，若其本身是正例，则TPR增加；若其本身是负例，则FPR增加，因此ROC曲线可以看作是随着阈值的不断移动，所有样本中正例与负例之间的“对抗”。

曲线越靠近左上角，意味着越多的正例优先于负例，模型的整体表现也就越好。

AUC曲线（Area Under the Curve）
对不同的ROC曲线进行比较的一个指标是曲线下的面积（Area Under Curve，AUC），曲线下面积（AUC）是评估中使用最广泛的指标之一。 它用于二分类问题。分类器的AUC等价于分类器随机选择正样本高于随机选择负样本的概率。 在定义AUC之前，让我们理解两个基本术语：

AUC（Area Under Curve）的值为ROC曲线下面的面积，若如上所述模型十分准确，则AUC为1。

但现实生活中尤其是工业界不会有如此完美的模型，一般AUC均在0.5到1之间，AUC越高，模型的区分能力越好

若AUC=0.5，即与上图中虚线重合，表示模型的区分能力与 随机猜测 没有差别。若AUC真的小于0.5，请检查一下是不是好坏标签标反了，或者是模型真的很差。


In [ ]:
PR (Precision Recall) 曲线
PR曲线展示的是Precision vs Recall的曲线，PR曲线与ROC曲线的相同点是都采用了TPR (Recall)，都可以用AUC来衡量分类器的效果。不同点是ROC曲线使用了FPR，而PR曲线使用了Precision，因此PR曲线的两个指标都聚焦于正例。
类别不平衡问题中由于主要关心正例，所以在此情况下PR曲线被广泛认为优于ROC曲线。
